<a href="https://colab.research.google.com/github/psygorozco/ML/blob/main/LANGCHAIN/Sistema_de_Respuesta_a_Preguntas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Cargar los datos de los "documentos largos".
* Dividir los documentos en secciones cortas llamadas fragmentos (chunks).
* Transformar los fragmentos en vectores númericos (Embeddings)
* Guardar los embeddings en una base de datos vectorial (Pinecone)
* Realizar las consultas

### Preparando los Datos

In [1]:
! pip install -r ./requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
! pip install pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.9 MB/s eta 0:00:00


In [3]:
! pip install docx2txt -q

  Preparing metadata (setup.py) ... done


In [4]:
! pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [5]:
import os
from dotenv import load_dotenv, find_dotenv

In [6]:
load_dotenv(find_dotenv(), override=True)

True

### Cargar Documentos

In [7]:
def cargar_documento(archivo):
    import os
    nombre, extension = os.path.splitext(archivo)
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Cargando {archivo}...')
        loader = PyPDFLoader(archivo)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Cargando {archivo}...')
        loader = Docx2txtLoader(archivo)
    else:
        print('El formato del documento no está soportado!')
        return None

    data = loader.load()
    return data

In [10]:
documento = "Minecraft.pdf"
contenido = cargar_documento(documento)
print(contenido)

Cargando Minecraft.pdf...
[Document(page_content='Minecraft  \n \nMinecraft  es un  videojuego  de construcción de tipo «mundo abierto » o sandbox  creado \noriginalmente  por el sueco  Markus Persson  (conocido comúnmente como «Notch») ,19 y \nposteriormente desarrollado por  Mojang Studios  (actualmente parte de  Microsoft ).20 Fue \nlanzado el 17 de mayo de 2009, y después de numerosos cam bios, su primera versión estable \n«1.0» fue publicada el 18 de noviembre de 2011.  \nMarkus Persson , el creador de Minecraft . \nUn mes antes del lanzamiento de su versión completa se estrenó una versión para \ndispositivos móviles llamada  Minecraft: Pocket Edition  en Android ,21 y el 17 de noviembre del \nmismo año fue lanzada la misma versión para  iOS.22 El 9 de mayo de 2012 fue lanzada la \nversión del  juego para  Xbox 360  y PS3. Todas las versiones de Minecraft  reciben \nactualizaciones constantes desde su lanzamiento. En octubre de 2014,  Minecraft  lanzó su \nedición para  PlayStatio

In [11]:
print(len(contenido))

7


In [12]:
# wikipedia
def desde_wikipedia(busqueda, lang='es', load_max_docs=3):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=busqueda, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [13]:
busqueda ="inteligencia artificial"
contenido = desde_wikipedia(busqueda)
print(contenido)

[Document(page_content='La inteligencia artificial  (IA), en el contexto de las ciencias de la computación, es una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos cuyo propósito es la creación de máquinas que imiten la inteligencia humana para realizar tareas, y que pueden mejorar conforme recopilen información.[1]\u200b[2]\u200b Se hizo presente poco después de la Segunda Guerra Mundial con el desarrollo de la «prueba de Turing», mientras que la locución fue acuñada en 1956 por el informático John McCarthy en la Conferencia de Dartmouth.\nEn la actualidad, la inteligencia artificial abarca una gran variedad de subcampos. Éstos van desde áreas de propósito general, aprendizaje y percepción, a otras más específicas como el Reconocimiento de voz, el juego de ajedrez, la demostración de teoremas matemáticos, la escritura de poesía y el diagnóstico de enfermedades. La inteligencia artificial sintetiza 

In [14]:
print(len(contenido))

3


### Fragmentar los datos

In [ ]:
def fragmentar(data, chunk_size=150):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=20)
    fragmentos = text_splitter.split_documents(data)
    return fragmentos

### Costos OpenAI

In [ ]:
def costo_embedding(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0001:.5f}')

### Borrando Index de Pinecone

In [ ]:
def borrar_indices(index_name='todos'):
    import pinecone
    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

    if index_name == 'todos':
        indexes = pinecone.list_indexes()
        print('Borrando todos los índices ... ')
        for index in indexes:
            pinecone.delete_index(index)
        print('Listo!')
    else:
        print(f'Borrando el índice: {index_name} ...', end='')
        pinecone.delete_index(index_name)
        print('Listo')

### Creando Vectores (Embeddings) y subirlos a (Pinecone)

In [ ]:
def creando_vectores(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings

    embeddings = OpenAIEmbeddings()

    pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'),
                  environment=os.environ.get('PINECONE_ENV'))

    if index_name in pinecone.list_indexes():
        print(f'El índice {index_name} ya existe. Cargando los embeddings ... ', end='')
        vectores = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creando el índice {index_name} y los embeddings ...', end='')
        pinecone.create_index(index_name, dimension=1536, metric='cosine')
        vectores = Pinecone.from_documents(fragmentos, embeddings, index_name=index_name)
        print('Ok')

    return vectores

### Haciendo consultas

In [ ]:
def consultas(vectores, pregunta):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vectores.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.run(pregunta)
    return answer

### Añadiendo memoria

In [ ]:
def consulta_con_memoria(vectores, pregunta, memoria=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(temperature=1)
    retriever = vectores.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    respuesta = crc({'question': pregunta, 'chat_history': memoria})
    memoria.append((pregunta, respuesta['answer']))

    return respuesta, memoria

### Resumen Final

In [ ]:
documento = "minecraft.pdf"
contenido = cargar_documento(documento)
fragmentos = fragmentar(contenido)
print(f"El Número de fragmentos es de: {len(fragmentos)} fragmentos")
costo_embedding(fragmentos)
borrar_indices("todos")
index_name = 'minecraft'
vectores = creando_vectores(index_name)

In [ ]:
memoria = []
while True:
    pregunta = input("Realiza una pregunta escribe 'salir' para terminar: \n")
    if pregunta == "salir":
        print("Adios!!!")
        break
    else:
        respuesta, memoria = consulta_con_memoria(vectores, pregunta, memoria)
        print(respuesta['answer'])


Realiza una pregunta escribe 'salir' para terminar: 
quien es el creador del juego?
El juego Minecraft: Story Mode fue desarrollado por Telltale Games en colaboración con Mojang Studios, la compañía que creó el juego original de Minecraft.
Realiza una pregunta escribe 'salir' para terminar: 
¿Qué me permite hacer el modo espectador?
El modo espectador te permite teletransportarte a otros jugadores en la partida. También te permite ver el juego desde el punto de vista de otro jugador o criatura. En algunos casos, puedes cambiar entre las perspectivas de primera y tercera persona. No hay información específica sobre otras funciones que pueda proporcionar el modo espectador.
